In [1]:
import webbrowser
import time
import json
import pandas as pd
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import REST, TimeFrame

In [2]:
payload=pd.read_html('https://en.wikipedia.org/wiki/Nasdaq-100')
df = payload[4]
#print(df['Ticker'])
symb_list=df['Ticker'].values.tolist()
#print(symb_list)

In [3]:
SEC_KEY = 'IH64H3nW41oQYp60wu2MWx2qM5DGH3SGitNqPIJb'
PUB_KEY = 'PKAAIHDW71UK328VXKX0'
BASE_URL = 'https://paper-api.alpaca.markets' # This is the base URL for paper trading
api = tradeapi.REST(key_id= PUB_KEY, secret_key=SEC_KEY, base_url=BASE_URL) # For real trading, don't enter a base_url


In [10]:
data=api.get_bars(symb_list, TimeFrame.Day, "2022-07-25",  "2022-07-29", limit=9999, adjustment='raw')
pd_data=data.df
#print(pd_data.head(10))


In [5]:
num_of_days=5
cash=100000
budget_limit=cash/10

In [6]:
subset=pd_data[['symbol','close']]
price_list=[]
symbol_list=[]
change_list=[]
quantity_list=[]
for i in range(len(symb_list)):
    dif=subset['close'][5*i+4]-subset['close'][5*i]
    avg=sum(subset['close'][5*i:5*i+5])/num_of_days
    symbol_list.append(subset['symbol'][5*i])
    price_list.append(subset['close'][5*i+4])
    change_list.append(dif/avg)
    quantity_list.append(int(budget_limit/subset['close'][5*i+4]))
#print(change_dict)
subset=pd.DataFrame(list(zip(symbol_list,price_list,change_list,quantity_list)),columns=['symb','price','change','quantity'])
#print(change_dict)
sorted_subset=subset.sort_values(by=['change'])

sorted_subset = sorted_subset.reset_index(drop=True)
print(sorted_subset.head())

    symb   price    change  quantity
0    PDD   49.01 -0.135196       204
1  CMCSA   37.52 -0.120444       266
2   CHTR  432.10 -0.101842        23
3   INTC   36.31 -0.073333       275
4     JD   59.50 -0.055967       168


In [43]:
import logging
logging.Logger.setLevel(level="INFO")

TypeError: setLevel() missing 1 required positional argument: 'self'

In [45]:
b="cba"
logging.warning(b)

In [ ]:
?api.submit_order

In [8]:
for index, row in sorted_subset.iterrows():
    if index<=5:
        api.submit_order(
  symbol=row["symb"], # Replace with the ticker of the stock you want to buy
  qty=row["quantity"],
  side='buy',
  type='limit', 
  limit_price=str(row["price"]),
  #notional='10000'
  #time_in_force='gtc' # Good 'til cancelled
)


In [ ]:
?api.submit_order

In [24]:
def place_order(sorted_subset):
    for index, row in sorted_subset.iterrows():
        if index<5:
            api.submit_order(
              symbol=row["symb"], # Replace with the ticker of the stock you want to buy
              qty=row["quantity"],
              side='buy',
              type='limit', 
              limit_price=str(row["price"]),
              #stop_price=str(row["price"]*1.001),
              time_in_force='day' # Good 'til cancelled
)
def cancel_all_order():
    api.cancel_all_orders()
    

In [ ]:
1/2

In [27]:
api.cancel_all_orders()
for i in range(5):
    place_order(sorted_subset)
    time.sleep(10)
    cancel_all_order()
    time.sleep(10)

In [ ]:
?api.cancel_all_orders

In [ ]:
orders=api.list_orders()
for i in range(len(orders)):
    order_dict=orders[i]._raw
    
    #print(order_dict['id'])
    api.cancel_order(order_dict['id'])
    time.sleep(5)

In [ ]:
api.cancel_order('e6c9fb45-f5ff-4e06-aa6a-f3fc8a4895c2')

In [ ]:
account=api.get_account()
print(account._raw)
#print(account)

In [34]:
orders=api.list_orders()
print(orders)
or

[Order({   'asset_class': 'us_equity',
    'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
    'canceled_at': None,
    'client_order_id': 'a31f81fa-1416-487d-883d-0b947ba7ad36',
    'created_at': '2022-08-01T19:56:38.207002Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': '54f0217f-aa60-4e7c-a23f-30bf8cf12db4',
    'legs': None,
    'limit_price': '200',
    'notional': None,
    'order_class': '',
    'order_type': 'limit',
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'source': 'access_key',
    'status': 'new',
    'stop_price': None,
    'submitted_at': '2022-08-01T19:56:38.206386Z',
    'subtag': None,
    'symbol': 'AAPL',
    'time_in_force': 'gtc',
    'trail_percent': None,
    'trail_price': None,
    'type': 'limit',
    'updated_at': '2022-08-01T19:56:38.215772Z'}), Order({

In [ ]:
account_history=[]
for i in range(30):
    account=api.get_account()
    account_history.append(account)
    time.sleep(10)
for account in account_history:
    print(account['equity'])
    

In [36]:
api.submit_order(
  symbol='AAPL', # Replace with the ticker of the stock you want to buy
  qty=1,
  side='sell',
  type='limit', 
  limit_price='100',
  time_in_force='gtc' # Good 'til cancelled
)

Order({   'asset_class': 'us_equity',
    'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
    'canceled_at': None,
    'client_order_id': '5dac03c0-4151-4a3b-a54e-523b47e7f690',
    'created_at': '2022-08-01T20:01:44.758173856Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': '891c7416-8254-4c19-8fca-fb781e162662',
    'legs': None,
    'limit_price': '100',
    'notional': None,
    'order_class': '',
    'order_type': 'limit',
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'source': None,
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2022-08-01T20:01:44.757446656Z',
    'subtag': None,
    'symbol': 'AAPL',
    'time_in_force': 'gtc',
    'trail_percent': None,
    'trail_price': None,
    'type': 'limit',
    'updated_at': '2022-08-01T20:01:44.758173856Z'})

In [37]:
history=api.get_trades('AAPL')

sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/trades 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/trades 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/trades 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/trades 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/trades 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/trades 3 more time(s)...


KeyboardInterrupt: 